In [1]:
import opendatasets as od
import numpy as np
import pandas as pd

In [18]:
from keras.models import Model
from keras.backend import mean,square,sum,shape,int_shape,random_normal,exp
from keras.layers import Input,Dense,Flatten,Conv2D,Conv2DTranspose,Lambda,Reshape,BatchNormalization,ReLU,LeakyReLU,Dropout,Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [3]:
od.download('https://www.kaggle.com/datasets/jessicali9530/celeba-dataset')

Skipping, found downloaded files in ".\celeba-dataset" (use force=True to force download)


In [3]:
data_path = 'G:\Generative-deep-learning\Projects\celeb-face-vae\celeba-dataset\img_align_celeba'

In [4]:
input_dim = (128,128,3)
batch_size = 32

In [5]:
img_gen = ImageDataGenerator(rescale=1./255)
flow = img_gen.flow_from_directory(data_path, target_size=input_dim[:2], batch_size=batch_size,shuffle=True,
                                   class_mode='input',subset='training')

Found 202599 images belonging to 1 classes.


In [6]:
# network params
input_shape = input_dim
kernel_size = 3
batch_size = 32
layers = [32,64,64,64]
latent_dim = 200


In [7]:
input_layer = Input(shape=input_shape,name='encoder_input_layer')

x = input_layer

for layer in layers:
    conv_layer = Conv2D(filters=layer, kernel_size=3,strides=2, padding='same')
    x = conv_layer(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.25)(x)
    

shape_before_flatten = int_shape(x)

x = Flatten()(x)

mu = Dense(units=latent_dim, name = 'mu')(x)
log_var = Dense(units=latent_dim,name = 'log_var')(x)

# encoder_mu_log_var =  Model(input_layer,(mu,log_var))

# encoder_mu_log_var.summary()


In [15]:
# print(shape(mu))

KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 200], name='tf.compat.v1.shape_5/Shape:0', description="created by layer 'tf.compat.v1.shape_5'")


In [16]:
def sampling(args):
    mu,log_var = args
    epsilon = random_normal(shape=shape(mu), mean=0, stddev=1.0)
    return mu + exp(log_var/2)*epsilon

In [17]:
z = Lambda(sampling,output_shape=(latent_dim,))([mu,log_var])

encoder = Model(input_layer,[mu,log_var,z])

encoder.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, 128, 128, 3  0          []                               
 r)                             )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   896         ['encoder_input_layer[0][0]']    
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

Non-trainable params: 448
__________________________________________________________________________________________________


In [11]:
# Decoder
decoder_temp_shape = np.prod(shape_before_flatten[1:])

decoder_input_layer = Input(shape=(latent_dim,),name='decoder_input')

x = Dense(decoder_temp_shape)(decoder_input_layer)
x = LeakyReLU()(x)
x = Reshape(shape_before_flatten[1:])(x)

for layer in layers[::-1]:
    conv_t_layer = Conv2DTranspose(filters=layer, kernel_size=3, strides=2, padding='same')
    x = conv_t_layer(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.25)(x)

decoder_output_layer = Activation('sigmoid')(x)

decoder = Model(decoder_input_layer,decoder_output_layer)

In [12]:
decoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 200)]             0         
                                                                 
 dense (Dense)               (None, 4096)              823296    
                                                                 
 reshape (Reshape)           (None, 8, 8, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 64)       36928     
 nspose)                                                         
                                                                 
 batch_normalization_4 (Batc  (None, 16, 16, 64)       256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 16, 16, 64)        0   

In [13]:
VAE = Model(input_layer,decoder(encoder_output))
VAE.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, 128, 128, 3  0          []                               
 r)                             )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   896         ['encoder_input_layer[0][0]']    
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                            

In [1]:
from keras.optimizers import Adam
from keras.losses import MeanSquaredError, KLDivergence

def net_loss(y,y_hat):
    r_loss = 10000*mean(square(y-y_hat),axis=[1,2,3])
    kl_loss = -0.5 * sum(1+log_var - square(mu) - exp(log_var))
    return r_loss + kl_loss
# def vae_sq_loss(y,y_hat,r_loss_factor):
#     r_loss = mean(square(y-y_hat),axis=[1,2,3])
#     return r_loss_factor*r_loss

# def vae_kl_loss(y,y_hat):
#     kl_loss = -0.5 * sum(1+log_var - square(mu) - exp(log_var),axis=1)
#     return kl_loss
optim = Adam(learning_rate=0.0005)

rloss_factor = 10000

VAE.compile(optimizer=optim, loss = 'mae')

NameError: name 'VAE' is not defined

In [15]:
# from keras.callbacks import LearningRateScheduler
# num_epochs = 100
# learning_rate = 0.0005
# def schedule(epochs,lr,decay=0.5):
#     new_lr = lr*(decay**epochs)
#     return new_lr


# lr_scheduler = LearningRateScheduler(schedule=schedule(num_epochs,learning_rate))

In [20]:

VAE.fit(flow,shuffle = True, epochs = 100)

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node 'mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "d:\condaa\envs\tflow2.9\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\condaa\envs\tflow2.9\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\pkdiv\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\pkdiv\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\pkdiv\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\pkdiv\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\condaa\envs\